# Node.js Streams homework

### Reference
* Short intro https://nodejs.dev/learn/nodejs-streams
* Node.js docs https://nodejs.org/api/stream.html

### Warning

If you encounter ```SyntaxError: Identifier <some-variable-name> has already been declared```
go to Kernel -> Restart in order to clear all declared variables.

### Task 1
You will have to apply the concepts presented at class in order to convert an audio file from a format to another without reading it entirely into memory.

This process is generically called transcoding, although sometimes transmuxing is a more accurate term. You will understand the difference after you solve task 2.

Since an image is worth a thousand words, here is what you'll actually do (in a streamed fashion):

![title](../img/transcoding.png)

### Solution

Please, don't change anything in the ```DON'T CHANGE``` sections of the code.

You can test that the conversion is successfully done by accessing http://localhost:5000/, where you have an audio player.

I suggest opening the url in an incognito session to prevent browser caching.

In [0]:
'use strict';

///////////// DON'T CHANGE /////////////
// check docs for https://www.npmjs.com/package/prism-media
const prism = require('prism-media');
///////////////////////////////////////
const fs = require('fs');
// You can add your dependencies here. 
// Note that you should use only Node.js builtin modules.
// aka no third-party dependencies allowed, like the one above.

///////////// DON'T CHANGE /////////////
const SOURCE_FILE = '../data/weekend.mp3';
const DESTINATION_FILE = '../www/static/audio.ogg';

const OUT_MUXER = 'ogg';
const OUT_SAMPLE_RATE = '48000';
const OUT_NUM_CHANNELS = '2';

const transcoder = new prism.FFmpeg({
  args: [
    '-analyzeduration', '0',
    '-loglevel', '0',
    '-f', OUT_MUXER,
    '-ar', OUT_SAMPLE_RATE,
    '-ac', OUT_NUM_CHANNELS,
  ],
});
///////////////////////////////////////

const input = fs.createReadStream(SOURCE_FILE);
const output = fs.createWriteStream(DESTINATION_FILE);
input.pipe(transcoder).pipe(output);

// You can add your logic here.
// Note that you should use only Node.js Stream APIs.
// aka reading of the whole file in memory and afterwards converting it is not scored.


### Task 2

Use the following code to find out what codec is used by the source file and by the destination file, respectively.

Also answer to the following questions:
1. What's the difference between container and codec?
2. What format do you think is more appropriate for live streaming conferences? (the one used by the source file, or the one used by the destionation file)

In [0]:
const { spawn } = require('child_process');
const ffprobe = require('ffprobe-static');

if (process.argv.length < 3) {
    console.error('Command format: node probe.js <path-to-media-file>');
}

const command = ffprobe.path;
const args = [
    '-v', 'error',
    '-select_streams', 'a:0',
    '-show_entries', 'stream=codec_long_name',
    '-of', 'default=noprint_wrappers=1:nokey=1',
    process.argv[2]
];

const subprocess = spawn(command, args, { stdio: 'inherit' });
subprocess.on('exit', (code, signal) => {

    if (code !== null) {

        if (code === 0) {
            console.log('Process exit with succes');
            return;
        }

        console.error(`Process exit code ${code}`);
        return;
    }

    if (signal !== null) {
        console.error(`Process killed with signal ${signal}`);
    }
});

// 1. The difference between container and codec lies in the fact that the container holds the data that the codec is going to compress/decompress when interacting with it.
// 2. Between the two formats mentioned above, MP3 (MPEG AUDIO LAYER 3) and Vorbis OGG, Vorbis is the better choice for live streaming since it has a higher speed (the destination file format).
// Ran the program with the following commands:
// node probe.js  ./data/weekend.mp3
// node probe.js ./www/static/audio.ogg